## UAV fleet Simulation 

This is for testing propouses

In [1]:
import numpy as np

# Custom modules
from helpers import local2global, plot_3d_interactive, kill_processes
from simulators import Simulator, Gazebo, QGC,Color
from plan import Plan, State
from oracle import Oracle

kill_processes()

## Create Plans

In [2]:
offsets = [  # east, north, up, heading
    (5, 5, 0, 90),
    (10, 0, 0, 45),
    (-5, -10, 0, 225),
    (-15, 0, 0, 0),
    (0, -20, 0, 0),
]
n_vehicles = len(offsets)
side_lens = (5, 7, 4, 1, 2)  # must agree with n_vehicles
local_paths = [
    Plan.create_square_path(side_len=side_len, alt=5) for side_len in side_lens
]
plans = [Plan.basic(wps=path, wp_margin=0.5) for path in local_paths]

## Visualization Parameters

In [3]:
homes = np.array([offset[:3] for offset in offsets])
global_paths = [
    local2global(np.array(path), home)
    for path, home in zip(local_paths, homes)
]
colors = [Color.GREEN, Color.BLUE,Color.RED, Color.ORANGE,Color.YELLOW]  # must agree with n_vehicles
waypoints = {
    f"waypoint_{i+1}": {"pos": global_paths[i], "color": colors[i]}
    for i in range(n_vehicles)
}
plot_3d_interactive(
    waypoints, title="Simulation Markers", expand=[0.2, 0.2, 0.6], ground=-0.05
)

## Choose Simulator

In [4]:
# simulator = Simulator(name="NONE", offsets=offsets, plans=plans)

# simulator = QGC(offsets=offsets, plans=plans, origin=(-35.3633245, 149.1652241, 0, 0))

simulator = Gazebo(
    offsets=offsets,
    plans=plans,
    world_path="simulators/gazebo/worlds/runway.world",
    models=n_vehicles * ["iris"],
    colors= colors,
    markers=waypoints,
)

Launch Simulator

In [5]:
uavs = simulator.launch(verbose=1)
orac = Oracle(uavs)

Vehicle 1 launched 🚀
Vehicle 2 launched 🚀
Vehicle 3 launched 🚀
Vehicle 4 launched 🚀
Vehicle 5 launched 🚀


## Execute Plan

In [6]:
while orac.vehs:
    orac.gather_broadcasts()
    for uav in orac.vehs.copy():
        orac.update_neighbors(uav)
        if uav.plan.state == State.DONE:
            orac.remove(uav)
        else:
            uav.act()
kill_processes()

Vehigcle 1 switched to mode: None
Vehicle 1: ▶️ Plan Started: 📋 basic
Vehicle 1: ▶️ Action Started: 🔧 PREARM
Vehicle 1: ▶️ Step Started: Check disarmed
Vehigcle 2 switched to mode: None
Vehicle 2: ▶️ Plan Started: 📋 basic
Vehicle 2: ▶️ Action Started: 🔧 PREARM
Vehicle 2: ▶️ Step Started: Check disarmed
Vehicle 3: ▶️ Plan Started: 📋 basic
Vehicle 3: ▶️ Action Started: 🔧 PREARM
Vehicle 3: ▶️ Step Started: Check disarmed
Vehicle 4: ▶️ Plan Started: 📋 basic
Vehicle 4: ▶️ Action Started: 🔧 PREARM
Vehicle 4: ▶️ Step Started: Check disarmed
Vehicle 5: ▶️ Plan Started: 📋 basic
Vehicle 5: ▶️ Action Started: 🔧 PREARM
Vehicle 5: ▶️ Step Started: Check disarmed
Vehicle 1: ✅ Step Done: Check disarmed
Vehicle 2: ✅ Step Done: Check disarmed
Vehicle 3: ✅ Step Done: Check disarmed
Vehicle 1: ▶️ Step Started: Check EKF
Vehicle 2: ▶️ Step Started: Check EKF
Vehicle 3: ▶️ Step Started: Check EKF
Vehicle 4: ✅ Step Done: Check disarmed
Vehicle 4: ▶️ Step Started: Check EKF
Vehicle 5: ✅ Step Done: Check disa